In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

パッケージの読み込み

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

データの読み込み

In [ ]:
data_train = pd.read_csv('../input/titanic/train.csv')
data_test = pd.read_csv('../input/titanic/test.csv')
data_gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')

EDA(探索的データ分析)
*データの特徴を把握した上で、適切なアルゴリズムを選定する必要がある

In [ ]:
#data_train

In [ ]:
#出力行数の確認
#pd.get_option('display.max_rows')
#pd.set_option('display.max_rows',900)

In [ ]:
#data_train

head()で表示行数を指定　デフォルトで5行

In [ ]:
#data_train.head()

In [ ]:
#data_gender_submission.head(10)

In [ ]:
#data_test.head(10)

#describeを用いれば、主要な統計指標もまとめて確認できる
以下ではデータの欠損や平均値及び中央値の特徴が表れている

In [ ]:
#data_train.describe()

In [ ]:
#データの概要を把握するには(pandas_profiling)が便利
#import pandas_profiling

In [ ]:
#実行はpandas.DataFrameのprofile_report()
#data_train.profile_report()

各特徴量(説明変数と同義、機械学習特有の言い回し)と目的変数の関係性を確認

In [ ]:
#PclassとSurvivedの件数を棒グラフで可視化
#sns.countplot(x = "Pclass",hue = "Survived", data = data_train)

In [ ]:
#sns.countplot(x = "Sex", hue = "Survived", data = data_train)

In [ ]:
#sns.countplot(x = "Age", hue ="Survived", data = data_train)

In [ ]:
#plt.hist(data_train.loc[data_train["Survived"] == 0, "Age"].dropna(),bins = 30, alpha = 0.7, label = "0")
#plt.hist(data_train.loc[data_train["Survived"] == 1, "Age"].dropna(),bins = 30, alpha = 0.7, label = "1")
#plt.xlabel("Age")
#plt.ylabel("Count")
#plt.legend(title ="Survived")

In [ ]:
#sns.countplot(x ="SibSp", hue = "Survived", data = data_train)

In [ ]:
#sns.countplot(x = "Parch", hue = "Survived", data = data_train)

In [ ]:
#plt.hist(data_train.loc[data_train["Survived"] == 0,"Fare"].dropna(),range(0,250),bins = 20,alpha = 0.7, label = "0")
#plt.hist(data_train.loc[data_train["Survived"] == 1,"Fare"].dropna(),range(0,250),bins = 20,alpha = 0.7, label = "1")
#plt.xlabel("Fare")
#plt.ylabel("Count")
#plt.legend(title = "Survived")

In [ ]:
#sns.countplot(x = "Embarked", hue = "Survived", data = data_train)

In [ ]:
#教師データとテストデータ、双方にエンジニアリングを行うため、いったん一つに結合する
data_all = pd.concat([data_train, data_test], sort = False)
data_all

In [ ]:
#各特徴量の欠損値の数を確認
#ここで、test_dataにはSurvivedのデータが存在しないため、結合後のデータフレームでは欠損値として扱われる

data_all.isnull().sum()

In [ ]:
#Sexの値を数値の置き換える　ここでは簡単にreplaceメソッドを用いる
#inplaceは変更の結果を元のデータフレームに反映させるか否かを指定する
data_all["Sex"].replace(["male","female"], [0,1], inplace=True)

In [ ]:
data_all.head

In [ ]:
#embarkedの欠損地を補完し、数値に置き換える
#本来はモデルの精度を比較して補完する必要がある
data_all["Embarked"].fillna("S",inplace = True)
data_all["Embarked"].replace(["S","C","Q"],[0, 1, 2], inplace = True)

In [ ]:
data_all

In [ ]:
#Fareの欠損値、Ageの欠損地を平均値で補完する　もともと数字なので変換の必要はない
#補完の方法として、欠損値があるデータ自体を削除したり、最頻値や中央値を用いたり、そもそも特徴量として採用しないことなどがある
data_all["Fare"].fillna(np.mean(data_all["Fare"]), inplace = True)
data_all["Age"].fillna(np.mean(data_all["Age"]), inplace = True)

In [ ]:
#影響が少ないと考えられる特徴量はいったん削除する
drop_columns = ["PassengerId","Name", "Parch", "SibSp", "Ticket", "Cabin"]
#axis=1で列方向に削除する。
data_all.drop(drop_columns, axis = 1, inplace = True)

In [ ]:
#欠損値ないか確認
data_all.isnull().sum()

In [ ]:
#結合したデータを教師データとテストデータに分離
#先頭から教師データ行数分のデータを抜き出している
data_train = data_all[:len(data_train)]
#教師データの行数分の１個下のデータから最後まで
data_test = data_all[len(data_train):]

モデル作成

In [ ]:
#多くのアルゴリズムは特徴量と目的変数を分離して投入
y_data_train = data_train["Survived"]
x_data_train = data_train.drop("Survived", axis = 1)
x_data_test = data_test.drop("Survived", axis = 1)

In [ ]:
#ランダムフォレストアルゴリズムをインポート
from sklearn.ensemble import RandomForestClassifier

#引数はそれぞれ決定木の数,木の深さ(階層),乱数
clf = RandomForestClassifier(n_estimators = 100, max_depth = 2, random_state = 0)

In [ ]:
#教師データの学習
clf.fit(x_data_train, y_data_train)

In [ ]:
#作成したモデルで予測
#predictの閾値は0.5が閾値なので,それを基準に0,1に変換
y_data_pred = clf.predict(x_data_test)

In [ ]:
y_data_pred

submitの作業
kaggleで用意されている提出用のフォーマットに従う必要あり

In [ ]:
#予測結果をsubmit用のcsvとして作成
submit = data_gender_submission
submit["Survived"] = list(map(int, y_data_pred))
submit.to_csv("randomForest_submit.csv", index = False)
#indexは保存しないという意